## Libraries

In [24]:
from pymongo import MongoClient
import pandas as pd
import sys

## Data for the collections

In [25]:
biology = pd.read_csv("Biology.csv")
philosophy = pd.read_csv("Philosophy.csv")
literature = pd.read_csv("Literature.csv")

In [26]:
#df to dict
data = biology.to_dict(orient = "records")
phil = philosophy.to_dict(orient = "records")
lit = literature.to_dict(orient = "records")

##  Connecting and creating the database and collections

In [27]:
client = MongoClient()
db = client.QuoteAPI

In [28]:
biology =  db.Biology
philosophy =  db.Philosophy
literature =  db.Literature

In [29]:
db.Biology.insert_many(data)
db.Philosophy.insert_many(phil)
db.Literature.insert_many(lit)    

In [30]:
#Para combinar todas las collecciones en una sola, donde solo tengamos la informacion de las frases y los autores
y = db.Biology.aggregate( [
   { '$unionWith': "Philosophy"},
   { '$unionWith': "Literature"},   
   { '$project': { '_id': 1, "Quote":1, "Author":1, "Category":1}},
   { '$out' : "Quote_Author" } ])